In [1]:
import pandas as pd
import numpy as np
from mca import diagsvd
import mca
import functools
import sklearn.model_selection as ms
from sklearn.preprocessing import StandardScaler as stdc
import gc
import _pickle as pickle

categorical = ['source_system_tab','source_screen_name','source_type','city','genre_ids','language']
numerical = ['song_length', 'song_year']

# Partitioning

In [2]:
file = 'df_clean.csv'
#categorical = ['source_system_tab','source_type','gender','city','genre_ids','language']
X = pd.read_csv('../Data/' + file,nrows=None, dtype={'target': np.uint8,'city':'category',
                                                     'source_system_tab':'category',
                                                     'source_type':'category','source_screen_name':'category',
                                                     'song_year':np.uint32,
                                                     'song_length':np.uint32,
                                                     'language':'category','genre_ids':'category'})
X = X.drop(columns='Unnamed: 0')
y = X['target']

X_num = X[numerical]
X_num.to_csv('X_num.csv')

del X_num
gc.collect()

print(X.dtypes)

X.head()

source_system_tab     category
source_screen_name    category
source_type           category
target                   uint8
city                  category
song_length             uint32
genre_ids             category
language              category
song_year               uint32
dtype: object


,source_system_tab,source_screen_name,source_type,target,city,song_length,genre_ids,language,song_year
0,explore,Explore,online-playlist,1,1,206471,359,52.0,2016
1,my library,Local playlist more,local-playlist,1,13,284584,1259,52.0,1999
2,my library,Local playlist more,local-playlist,1,13,225396,1259,52.0,2006
3,my library,Local playlist more,local-playlist,1,13,255512,1019,-1.0,2010
4,explore,Explore,online-playlist,1,1,187802,1011,52.0,2016


In [ ]:

# for c in categorical:
#     print(pd.crosstab(index=X["target"],columns=X[c]))


Particiono

In [3]:
DummiesX = pd.get_dummies(data=X[categorical])

print(DummiesX.shape)
DummiesX.head()

(6317409, 219)


,source_system_tab_discover,source_system_tab_explore,source_system_tab_listen with,source_system_tab_my library,source_system_tab_notification,source_system_tab_radio,source_system_tab_search,source_system_tab_settings,source_screen_name_Album more,source_screen_name_Artist more,...,language_-1.0,language_10.0,language_17.0,language_24.0,language_3.0,language_31.0,language_38.0,language_45.0,language_52.0,language_59.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
from gensim import corpora, models, similarities
from gensim.corpora import SvmLightCorpus
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn import metrics

In [ ]:
num_topics = 50
lsa = TruncatedSVD(50,algorithm='randomized')
dtm_lsa = lsa.fit_transform(DummiesX)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)


In [31]:
proportion = 0.3
(X_train, X_test, y_train, y_test) = ms.train_test_split(dtm_lsa, y, test_size=proportion, random_state=23)


In [34]:
print(lsa.explained_variance_ratio_.sum())  

1.0914944800898871


In [32]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train.values.ravel())
predicted = lda.predict(X_test)
acc = metrics.accuracy_score(y_test, predicted)
print("Accuracy:",acc)

print('Full report: \n', metrics.classification_report(y_test, predicted))


Accuracy: 0.6260957153854718
Full report: 
              precision    recall  f1-score   support

          0       0.62      0.62      0.62    933345
          1       0.63      0.63      0.63    961878

avg / total       0.63      0.63      0.63   1895223



In [ ]:
del X
gc.collect()

Elimino las columnas que tienen cero para todas las filas de algun subconjunto

In [ ]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)


In [ ]:
proportion = 0.4
(X_train, X_test, y_train, y_test) = ms.train_test_split(DummiesX, y, test_size=proportion, random_state=23)


(X_MCA, X_testM, y_train, y_test) = ms.train_test_split(X_train, y_train, test_size=0.99, random_state=23)

size = [X_MCA.shape[0], X_MCA.shape[1]]


X_MCA_sum = (X_MCA.values.sum(axis=0) != 0)
X_test_sum = (X_test.values.sum(axis=0) != 0)
index = np.logical_and(X_MCA_sum, X_test_sum)
X_train = X_train.loc[:, index]
X_MCA = X_MCA.loc[:, index]
X_test = X_test.loc[:, index]


# X_train_sum = (X_train.values.sum(axis=1) != 0)
# X_test_sum = (X_test.values.sum(axis=1) != 0)
# index = np.logical_and(X_train_sum, X_test_sum)
# X_train = X_train.loc[:, index]
# X_test = X_test.loc[:, index]

idx = X_train.index

X_test['target'] = y_test
X_test.to_pickle('X_test.pkl')
del DummiesX, X_test
gc.collect()
                      

In [ ]:

gc.collect()
  

In [ ]:
X_train.shape, X_MCA.shape,
# size

# MCA

In [ ]:
N_all = np.sum(X_MCA.values[0:size[0],:])

Z_residual = X_MCA.values[0:size[0],:] / N_all
X_train.to_pickle('X_train.pkl')
del X_train
gc.collect()
Z_residual.shape

In [ ]:
Sum_r = np.sum(Z_residual, axis=1)
Sum_c = np.sum(Z_residual, axis=0)
print(size, Sum_r.shape, Sum_c.shape, N_all)

In [ ]:
Z_residual -= np.outer(Sum_r, Sum_c)


In [ ]:
D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
del Sum_r
gc.collect()
D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
del Sum_c
gc.collect()

In [ ]:
MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)

In [ ]:
P, S, Q = np.linalg.svd(MCA_mat)
del MCA_mat
gc.collect()

In [ ]:
Lam = S ** 2
Expl_var = Lam / np.sum(Lam)

# print('Eigen values are ', Lam)
# print('Explained variance of eigen vectors are ', Expl_var)

data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var)}
columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'
np.round(table2.astype(float), 4)
table2.to_csv('table2.csv')
del Lam, Expl_var
gc.collect()

In [ ]:
columns_keep = len(table2['τI'])
while(table2['τI'][0:columns_keep].sum()>0.9):
    columns_keep = columns_keep - 2
    

print('Varianza explicada tomando '+str(columns_keep)+' de '+ str(len(table2['τI']))+' features: ', table2['τI'][0:columns_keep].sum())

In [ ]:
S.shape
# size

In [ ]:
S_d = diagsvd(S, size[0], size[1])


In [ ]:
F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
del P,D_r_sqrt_mi
gc.collect()

G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows


In [ ]:
del  Q, S_d, D_c_sqrt_mi
gc.collect()

In [ ]:

X_cat_trainMatrix= Matrix_mult(pd.read_pickle('X_train.pkl').values, G[:, :columns_keep]) / S[:columns_keep] / 10


In [ ]:
names = ['V' + str(i) for i in range(columns_keep)]
X_cat_train = pd.DataFrame(X_cat_trainMatrix, columns=names, index = idx)

del X_cat_trainMatrix
gc.collect()

X_num = pd.read_csv('X_num.csv',nrows=None, dtype={'song_year':np.uint64,
                                                     'song_length':np.uint64,
                                                     })

X_num = pd.DataFrame(stdc().fit_transform(X_num.values), columns=X_num.columns, index = X_num.index)
X_num = X_num.drop(columns=['Unnamed: 0'])
(X_train_num, X_test_num, y_train, y_test) = ms.train_test_split(X_num, y, test_size=proportion, random_state=23)

concat_Train = pd.concat([X_cat_train, X_train_num], axis=1, ignore_index=False)
concat_Train['target'] = y_train
concat_Train.to_csv('../Data/preprocessedTrainMCA.csv')

del X_cat_train,X_train_num
gc.collect()

In [ ]:
concat_Train.head()


In [ ]:
X_cat_test= pd.DataFrame(Matrix_mult(pd.read_pickle('X_test.pkl').drop(columns='target')
.values, G[:, :columns_keep]) / S[:columns_keep] / 10, columns=names, index =pd.read_pickle('X_test.pkl').index)

concat_Test = pd.concat([DummiesX, X_test_num], axis=1, ignore_index=False)
concat_Test['target'] = y_test
concat_Test.to_csv('../Data/preprocessedTestMCA.csv')
concat_Test.isnull().sum().sum()